In [6]:
import torch
from sklearn.manifold import TSNE
from torchvision.models.feature_extraction import create_feature_extractor
import matplotlib.pyplot as plt
from matplotlib import patches
import torch.nn as nn
from inference import MULSAInference

ModuleNotFoundError: No module named 'models'

In [ ]:

# model_path = "checkpoint-model-epoch15.pth" ## load the model
# model = ResNet(20)
# model= torch.load(model_path)
model_path = "/home/hello-robot/soundsense/soundsense/stretch/models/baselines/mulsa/weights/unimodal/simple_task/03-28-16:48:44-jobid=0-epoch=2-step=2634.ckpt"
model = MULSAInference.load_from_checkpoint(model_path)
model.eval()

feature_extractor_model = nn.Sequential(*list(model.children())[:-1])
model = feature_extractor_model.to('cuda')

model.eval()

In [ ]:

test_loader = get_data_loader('voc', train=False, batch_size=1, split='test', inp_size=224)

outputs_list = []
targets_list = []
indices = []

for i, (img, target, _) in enumerate(test_loader):
    if i > 999:
        break
    with torch.no_grad():
        output = model(img.to('cuda'))
    outputs_list.append(output)
    targets_list.append(target)
    indices.append(i)

outputs = torch.cat(outputs_list, dim=0)
targets = torch.cat(targets_list, dim=0)

In [ ]:
outputs = outputs.view(outputs.size(0), -1).cpu().numpy()
targets = targets.cpu().numpy()

In [ ]:
tsne = TSNE(n_components=2)
features_tsne = tsne.fit_transform(outputs)
features_tsne.shape

In [ ]:
class_colors = plt.cm.tab20(np.arange(20))

plt.figure(figsize=(10, 8))
for i in range(20):
    indices = np.where(targets[:, i] == 1)[0]
    plt.scatter(features_tsne[indices, 0], features_tsne[indices, 1], label=f"Class {i}", color=class_colors[i])

plt.title('t-SNE Projection of Features with Ground Truth Labels')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend(title='Object Class', bbox_to_anchor=(1.05, 1), loc='upper left')  